# `Mounting Dataset From Drive.` 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# `1.Import Libraries.`

In [ ]:
# Import Libraries
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)

# Keras API
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D,Activation,AveragePooling2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

# `2.Load Data into Train and Test Variables.`

In [ ]:
# My data is in google drive.
train_dir ="drive/My Drive/Train/"
test_dir="drive/My Drive/Test/"

# `3.Function To count Images In Each Data Set.`

In [ ]:
# function to get count of images
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count

In [ ]:
# Preprocessing data.
train_datagen = ImageDataGenerator( rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2, # Random zoom
                                   validation_split=0.2, # validation split 20%.
                                   horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1./255)

# `4.CNN Parameter Building.`

In [ ]:
train_samples = get_files(train_dir)
 
num_classes = len(glob.glob(train_dir+"/*"))
 
test_samples=get_files(test_dir) 
 
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

In [ ]:
# CNN building.
model = Sequential()
 
model.add(Conv2D(32, (5, 5),input_shape=input_shape,activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
 
model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))   
 
model.add(Flatten()) # Flatten 
 
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))
 
model.add(Dense(128,activation='relu'))          
model.add(Dense(num_classes,activation='softmax'))
 
model.summary()

In [ ]:
# set height and width and color of input image.
img_width,img_height =256,256
input_shape=(img_width,img_height,3)
batch_size =32
 
train_generator =train_datagen.flow_from_directory(train_dir,
                                                   target_size=(img_width,img_height),
                                                   batch_size=batch_size)
 
test_generator=test_d hiatagen.flow_from_directory(test_dir,shuffle=True,
                                                   target_size=(img_width,img_height),
                                                   batch_size=batch_size)

In [ ]:
model_layers = [ layer.name for layer in model.layers]
print('layer name : ',model_layers)

# `6.Training The Model.`

In [ ]:
# validation data.
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size) 

In [ ]:
# Model building to get trained with parameters.
opt=keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
train = model.fit_generator(train_generator,
                          epochs=15,
                          steps_per_epoch=train_generator.samples // batch_size,
                          validation_data=validation_generator,
                          validation_steps= validation_generator.samples// batch_size,verbose=1)


# `7.Plot For Accuracy And Losses.`

In [ ]:
acc = train.history['acc']
val_acc = train.history['val_acc']
loss = train.history['loss']
val_loss = train.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy 
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()
 
plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

# `8. Evaluate model using unseen data.`


In [ ]:
score,accuracy =model.evaluate(test_generator,verbose=1)
print("Test score is {}".format(score))
print("Test accuracy is {}".format(accuracy))

# `9.Saving Model.` 

In [ ]:
# Save entire model with optimizer, architecture and training configuration.
from keras.models import load_model
model.save('seed.h5')

# `10.Load Model.`

In [ ]:
# Loading model and predict.
from keras.models import load_model
model=load_model('seed.h5')
 
Classes = ["good","bad","worst","excellent","average","good","bad","worst","excellent"]

In [ ]:
good
bad
worst
excellent
average
Good
Bad
Worst
Excellent
Average

# `11.Time For Predictions.`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Pre-Processing test data same as train data.
img_width=256
img_height=256
#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

from keras.preprocessing import image

def prepare(img_path):
    img = image.load_img(img_path, target_size=(256, 256))
    x = image.img_to_array(img)
    x = x/255
    return np.expand_dims(x, axis=0)
    
    
result = model.predict_classes([prepare('/content/drive/My Drive/Test/seed/dc09c4e1-8b58-40e4-a2f2-61b3f19eff94___Matt.S_CG 1010.JPG')])
disease=image.load_img('/content/drive/My Drive/Test/seed/dc09c4e1-8b58-40e4-a2f2-61b3f19eff94___Matt.S_CG 1010.JPG')
plt.imshow(disease)
print (Classes[int(result)])

# `12.Convert Model To "tflite format."`.
- This conversion is done because to make our model interpertable with App.
- tflite is tensorflowlite made for mobile versions.

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model_file( 'seed.h5' )
model = converter.convert()
file = open( 'model.tflite' , 'wb' )
file.write( model )